In [ ]:
#%pip install git+https://github.com/huggingface/peft
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl 
#%pip install https://pypi.org/simple/ bitsandbytes

In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Process ForkProcess-20:
Process ForkProcess-19:
Process ForkProcess-29:
Process ForkProcess-26:
Process ForkProcess-27:
Process ForkProcess-13:
Process ForkProcess-21:
Process ForkProcess-16:
Process ForkProcess-23:
Process ForkProcess-12:
Process ForkProcess-6:
Process ForkProcess-17:
Process ForkProcess-24:
Process ForkProcess-28:
Process ForkProcess-7:
Process ForkProcess-18:
/usr/local/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multit

In [ ]:
config = PeftConfig.from_pretrained("FO-UA/adapt-llm-Misc-Fr-280Xr32Xa64")
model_new = AutoModelForCausalLM.from_pretrained("AdaptLLM/finance-chat")
model_new = PeftModel.from_pretrained(model_new, "FO-UA/adapt-llm-Misc-Fr-280Xr32Xa64")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("FO-UA/adapt-llm-Misc-Fr-280Xr32Xa64", trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

In [2]:
config = PeftConfig.from_pretrained("FO-UA/adapt-llm-Misc-Fr-1")
model_new = AutoModelForCausalLM.from_pretrained("AdaptLLM/finance-chat")
model_new = PeftModel.from_pretrained(model_new, "FO-UA/adapt-llm-Misc-Fr-1")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("FO-UA/adapt-llm-Misc-Fr-1", trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.15s/it]
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/us

(True, True)

# Inference

In [3]:
# Put your input here:
user_input = '''

Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID":"BaseGen1",
"Seniority_previous_company(N-2) (in days)":0,
"Seniority_previous_company(N-1) (in days)":0,
"Seniority_in_Group (in days)":11000,
"Contract_Date":"18/11/2020",
"Leave_Date":"31/12/2050",
"Entry_Date_Company_1":"01/01/1753",
"Sex":"Mr",
"Contract Type":"DI",
"CODENT":"ReRecruited",
"Classification":"CAS",
"Job Title":"Assimilated Executive",
"Specific_title":"Assimilated Executive",
"Work_form":"Part Time",
"Entry_Category":"ACTIVE",
"CODSOR":"Currently IN",
"Original_company":"TMG",
"Leave_Category":"Unspecified",
"Leave_Reason":"Currently IN",
"Transfer_Reason":"Not Transferred",
"Transfer_company":"UNSPECFIED",
"Explanation":"Contract Entry Date is 2020, he\'s been in the company for 4 years (2024-2020),  he\'s a classified an Assimilated Executive,the rank associated to him is higher than that of his actual seniority years and experience,"

Instruction : Generate 3 Misclassification Fraud scenarios :
Completion : 
'''

#our_system_prompt="\nWe are trying to generate accounting Fraud data, in different fraud types : Employee Misclassification Fraud, Ghost Fraud, and Timesheet Fraud data, based on the examples above, try to generate one example of Misclassification fraud.\n"
initial_prompt = '''A Misclassification fraud is an payroll fraud type where the perpetrator misclassifies an employee of an enterprise in order for the employee to get higher or lower pay than what his diploma or seniority should allow him.
Note that the actual year we are currently at is : 2024. '''

prompt = f"<s>[INST] <<SYS>>{initial_prompt}<</SYS>>\n\n{user_input} [/INST]"


inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids.to(model_new.device)
outputs =model_new.generate(input_ids=inputs, 
                             #max_length=4096, #32768  #16384  #8192  #4096
                             max_new_tokens=1500,  #1200 for 3 examples 
                             do_sample=True,
                             temperature=1.4
                            )[0]

answer_start = int(inputs.shape[-1])
pred = tokenizer.decode(outputs[answer_start:], skip_special_tokens=True)

print(f'### User Input:\n{user_input}\n\n### Assistant Output:\n{pred}')


### User Input:


Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID":"BaseGen1",
"Seniority_previous_company(N-2) (in days)":0,
"Seniority_previous_company(N-1) (in days)":0,
"Seniority_in_Group (in days)":11000,
"Contract_Date":"18/11/2020",
"Leave_Date":"31/12/2050",
"Entry_Date_Company_1":"01/01/1753",
"Sex":"Mr",
"Contract Type":"DI",
"CODENT":"ReRecruited",
"Classification":"CAS",
"Job Title":"Assimilated Executive",
"Specific_title":"Assimilated Executive",
"Work_form":"Part Time",
"Entry_Category":"ACTIVE",
"CODSOR":"Currently IN",
"Original_company":"TMG",
"Leave_Category":"Unspecified",
"Leave_Reason":"Currently IN",
"Transfer_Reason":"Not Transferred",
"Transfer_company":"UNSPECFIED",
"Explanation":"Contract Entry Date is 2020, he's been in the company for 4 years (2024-2020),  he's a classified an Assimilated Executive,the rank associated to him is higher than that of his actual seniority years and experience,"

Instruction : Generate 3 Mi

In [5]:
# save str in file format
with open( 'Assisant_output1.txt', 'w') as f:
    f.write(pred)
print("file saved")

# save to csv


file saved


In [6]:
# Put your input here:
user_input = '''

Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID":"BaseGen1",
"Seniority_previous_company(N-2) (in days)":0,
"Seniority_previous_company(N-1) (in days)":0,
"Seniority_in_Group (in days)":11000,
"Contract_Date":"18/11/2020",
"Leave_Date":"31/12/2050",
"Entry_Date_Company_1":"01/01/1753",
"Sex":"Mr",
"Contract Type":"DI",
"CODENT":"ReRecruited",
"Classification":"CAS",
"Job Title":"Assimilated Executive",
"Specific_title":"Assimilated Executive",
"Work_form":"Part Time",
"Entry_Category":"ACTIVE",
"CODSOR":"Currently IN",
"Original_company":"TMG",
"Leave_Category":"Unspecified",
"Leave_Reason":"Currently IN",
"Transfer_Reason":"Not Transferred",
"Transfer_company":"UNSPECFIED",
"Explanation":"Contract Entry Date is 2020, he\'s been in the company for 4 years (2024-2020),  he\'s a classified an Assimilated Executive,the rank associated to him is higher than that of his actual seniority years and experience,"

Instruction : Generate 3 Misclassification Fraud scenarios :
Completion : 

'''

#our_system_prompt="\nWe are trying to generate accounting Fraud data, in different fraud types : Employee Misclassification Fraud, Ghost Fraud, and Timesheet Fraud data, based on the examples above, try to generate one example of Misclassification fraud.\n"
initial_prompt = '''A Misclassification fraud is an payroll fraud type where the perpetrator misclassifies an employee of an enterprise in order for the employee to get higher or lower pay than what his diploma or seniority should allow him.
Note that the actual year we are currently at is : 2024. '''

prompt = f"<s>[INST] <<SYS>>{initial_prompt}<</SYS>>\n\n{user_input} [/INST]"


inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids.to(model_new.device)
outputs =model_new.generate(input_ids=inputs, 
                             #max_length=4096, #32768  #16384  #8192  #4096
                             max_new_tokens=2000,  #1200 for 3 examples 
                             do_sample=True,
                             temperature=1.4
                            )[0]

answer_start = int(inputs.shape[-1])
pred = tokenizer.decode(outputs[answer_start:], skip_special_tokens=True)

print(f'### User Input:\n{user_input}\n\n### Assistant Output:\n{pred}')


### User Input:


Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID":"BaseGen1",
"Seniority_previous_company(N-2) (in days)":0,
"Seniority_previous_company(N-1) (in days)":0,
"Seniority_in_Group (in days)":11000,
"Contract_Date":"18/11/2020",
"Leave_Date":"31/12/2050",
"Entry_Date_Company_1":"01/01/1753",
"Sex":"Mr",
"Contract Type":"DI",
"CODENT":"ReRecruited",
"Classification":"CAS",
"Job Title":"Assimilated Executive",
"Specific_title":"Assimilated Executive",
"Work_form":"Part Time",
"Entry_Category":"ACTIVE",
"CODSOR":"Currently IN",
"Original_company":"TMG",
"Leave_Category":"Unspecified",
"Leave_Reason":"Currently IN",
"Transfer_Reason":"Not Transferred",
"Transfer_company":"UNSPECFIED",
"Explanation":"Contract Entry Date is 2020, he's been in the company for 4 years (2024-2020),  he's a classified an Assimilated Executive,the rank associated to him is higher than that of his actual seniority years and experience,"

Instruction : Generate 3 Mi

In [7]:
# save str in file format
with open( 'Assisant_output2.txt', 'w') as f:
    f.write(pred)
print("file saved")

# save to csv


file saved


In [8]:
# Put your input here:
user_input = '''

Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID":"BaseGen1",
"Seniority_previous_company(N-2) (in days)":0,
"Seniority_previous_company(N-1) (in days)":0,
"Seniority_in_Group (in days)":11000,
"Contract_Date":"18/11/2020",
"Leave_Date":"31/12/2050",
"Entry_Date_Company_1":"01/01/1753",
"Sex":"Mr",
"Contract Type":"DI",
"CODENT":"ReRecruited",
"Classification":"CAS",
"Job Title":"Assimilated Executive",
"Specific_title":"Assimilated Executive",
"Work_form":"Part Time",
"Entry_Category":"ACTIVE",
"CODSOR":"Currently IN",
"Original_company":"TMG",
"Leave_Category":"Unspecified",
"Leave_Reason":"Currently IN",
"Transfer_Reason":"Not Transferred",
"Transfer_company":"UNSPECFIED",
"Explanation":"Contract Entry Date is 2020, he\'s been in the company for 4 years (2024-2020),  he\'s a classified an Assimilated Executive,the rank associated to him is higher than that of his actual seniority years and experience,"

Instruction : Generate 3 Misclassification Fraud scenarios :
Completion : 
'''

#our_system_prompt="\nWe are trying to generate accounting Fraud data, in different fraud types : Employee Misclassification Fraud, Ghost Fraud, and Timesheet Fraud data, based on the examples above, try to generate one example of Misclassification fraud.\n"
initial_prompt = '''A Misclassification fraud is an payroll fraud type where the perpetrator misclassifies an employee of an enterprise in order for the employee to get higher or lower pay than what his diploma or seniority should allow him.
Note that the actual year we are currently at is : 2024. '''

prompt = f"<s>[INST] <<SYS>>{initial_prompt}<</SYS>>\n\n{user_input} [/INST]"


inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids.to(model_new.device)
outputs =model_new.generate(input_ids=inputs, 
                             #max_length=4096, #32768  #16384  #8192  #4096
                             max_new_tokens=1500,  #1200 for 3 examples 
                             do_sample=True,
                             temperature=1.2
                            )[0]

answer_start = int(inputs.shape[-1])
pred = tokenizer.decode(outputs[answer_start:], skip_special_tokens=True)

print(f'### User Input:\n{user_input}\n\n### Assistant Output:\n{pred}')


### User Input:


Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID":"BaseGen1",
"Seniority_previous_company(N-2) (in days)":0,
"Seniority_previous_company(N-1) (in days)":0,
"Seniority_in_Group (in days)":11000,
"Contract_Date":"18/11/2020",
"Leave_Date":"31/12/2050",
"Entry_Date_Company_1":"01/01/1753",
"Sex":"Mr",
"Contract Type":"DI",
"CODENT":"ReRecruited",
"Classification":"CAS",
"Job Title":"Assimilated Executive",
"Specific_title":"Assimilated Executive",
"Work_form":"Part Time",
"Entry_Category":"ACTIVE",
"CODSOR":"Currently IN",
"Original_company":"TMG",
"Leave_Category":"Unspecified",
"Leave_Reason":"Currently IN",
"Transfer_Reason":"Not Transferred",
"Transfer_company":"UNSPECFIED",
"Explanation":"Contract Entry Date is 2020, he's been in the company for 4 years (2024-2020),  he's a classified an Assimilated Executive,the rank associated to him is higher than that of his actual seniority years and experience,"

Instruction : Generate 3 Mi

In [9]:
# save str in file format
with open( 'Assisant_output3.txt', 'w') as f:
    f.write(pred)
print("file saved")

# save to csv


file saved


In [11]:
# Put your input here:
user_input = '''

Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID": BaseGen11,
"Seniority_previous_company(N-2) (in days)":0,
"Contract_in_group_duration":11135 days,
"Seniority_previous_company(N-1) (in days)":0,
"Seniority_in_Group (in days)":12135,
"Contract_Date":10/10/2017,
"Leave_Date":31/12/2050,
"Entry_Date_Company_1":01/01/1753,
"Sex":Mr,
"Contract Type":DI,
"CODENT":New Recruit,
"Classification":DIR,
"Job Title":Manager/Director,
"Specific_title":Executive C3,
"Work_form":Part Time,
"Entry_Category":ACTIVE,
"CODSOR": Currently IN,
"Original_company": EUR,
"Leave_Category":Unspecified,
"Leave_Reason":Currently IN,
"Transfer_Reason":Not Transferred,
"Transfer_company":"UNSPECFIED,
"Explanation":"Contract Entry Date is 2017, he's been in the company for 7 years (2024-2017), yet, he's a classified a Manager, he should be higher in the ranks,"

Instruction : Generate 5 Misclassification Fraud scenarios :
Completion : 
'''

#our_system_prompt="\nWe are trying to generate accounting Fraud data, in different fraud types : Employee Misclassification Fraud, Ghost Fraud, and Timesheet Fraud data, based on the examples above, try to generate one example of Misclassification fraud.\n"
initial_prompt = '''A Misclassification fraud is an payroll fraud type where the perpetrator misclassifies an employee of an enterprise in order for the employee to get higher or lower pay than what his diploma or seniority should allow him.
Note that the actual year we are currently at is : 2024. '''

prompt = f"<s>[INST] <<SYS>>{initial_prompt}<</SYS>>\n\n{user_input} [/INST]"


inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids.to(model_new.device)
outputs =model_new.generate(input_ids=inputs, 
                             #max_length=4096, #32768  #16384  #8192  #4096
                             max_new_tokens=2500,  #1200 for 3 examples 
                             do_sample=True,
                             temperature=1.2
                            )[0]

answer_start = int(inputs.shape[-1])
pred = tokenizer.decode(outputs[answer_start:], skip_special_tokens=True)

print(f'### User Input:\n{user_input}\n\n### Assistant Output:\n{pred}')


### User Input:


Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID": BaseGen11,
"Seniority_previous_company(N-2) (in days)":0,
"Contract_in_group_duration":11135 days,
"Seniority_previous_company(N-1) (in days)":0,
"Seniority_in_Group (in days)":12135,
"Contract_Date":10/10/2017,
"Leave_Date":31/12/2050,
"Entry_Date_Company_1":01/01/1753,
"Sex":Mr,
"Contract Type":DI,
"CODENT":New Recruit,
"Classification":DIR,
"Job Title":Manager/Director,
"Specific_title":Executive C3,
"Work_form":Part Time,
"Entry_Category":ACTIVE,
"CODSOR": Currently IN,
"Original_company": EUR,
"Leave_Category":Unspecified,
"Leave_Reason":Currently IN,
"Transfer_Reason":Not Transferred,
"Transfer_company":"UNSPECFIED,
"Explanation":"Contract Entry Date is 2017, he's been in the company for 7 years (2024-2017), yet, he's a classified a Manager, he should be higher in the ranks,"

Instruction : Generate 5 Misclassification Fraud scenarios :
Completion : 


### Assistant Output:
1

In [12]:
# save str in file format
with open( 'Assisant_output4.txt', 'w') as f:
    f.write(pred)
print("file saved")

# save to csv


file saved


In [13]:
def batch_generation(L,nb_per_batch=5,max_new_tok=2500,temp=1.2) :
    L_result=[]
    for i in range(len(L)) :
        user_input=L[i]+'''
        
        Instruction : Generate 5 Misclassification Fraud scenarios :
        Completion : 

        '''
        initial_prompt = '''A Misclassification fraud is a payroll fraud type where the perpetrator misclassifies an employee of an enterprise in order for the employee to get higher or lower pay than what his diploma or seniority should allow him.
        Note that the actual year we are currently at is : 2024. '''

        prompt = f"<s>[INST] <<SYS>>{initial_prompt}<</SYS>>\n\n{user_input} [/INST]"


        inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids.to(model_new.device)
        outputs =model_new.generate(input_ids=inputs, 
                             max_new_tokens=2500,  #1200 for 3 examples 
                             do_sample=True,
                             temperature=1.2
                            )[0]

        answer_start = int(inputs.shape[-1])
        pred = tokenizer.decode(outputs[answer_start:], skip_special_tokens=True)
        L_result.append(pred)
        
        # save str in file format
        with open( f'Assisant_output{i}.txt', 'w') as f:
            f.write(pred)
        print(f'file {i} saved')

        print(f'step{i} done')
    
    return L_result


In [14]:
L=['''
    Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID":"BaseGen1",
"Seniority_previous_company(N-2) (in days)":0,
"Contract_in_group_duration":"12000 days",
"Seniority_previous_company(N-1) (in days)":0,
"Seniority_in_Group (in days)":11000,
"Contract_Date":"18/11/2020",
"Leave_Date":"31/12/2050",
"Entry_Date_Company_1":"01/01/1753",
"Sex":"Mr",
"Contract Type":"DI",
"CODENT":"ReRecruited",
"Classification":"CAS",
"Job Title":"Assimilated Executive",
"Specific_title":"Assimilated Executive",
"Work_form":"Part Time",
"Entry_Category":"ACTIVE",
"CODSOR":"Currently IN",
"Original_company":"TMG",
"Leave_Category":"Unspecified",
"Leave_Reason":"Currently IN",
"Transfer_Reason":"Not Transferred",
"Transfer_company":"UNSPECFIED",
"Explanation":"Contract Entry Date is 2020, he\'s been in the company for 4 years (2024-2020),  he\'s a classified an Assimilated Executive,the rank associated to him is higher than that of his actual seniority years and experience,"
''',
   '''
Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID": BaseGen11,
"Seniority_previous_company(N-2) (in days)":0,
"Contract_in_group_duration":11135 days,
"Seniority_previous_company(N-1) (in days)":0,
"Seniority_in_Group (in days)":12135,
"Contract_Date":10/10/2017,
"Leave_Date":31/12/2050,
"Entry_Date_Company_1":01/01/1753,
"Sex":Mr,
"Contract Type":DI,
"CODENT":New Recruit,
"Classification":DIR,
"Job Title":Manager/Director,
"Specific_title":Executive C3,
"Work_form":Part Time,
"Entry_Category":ACTIVE,
"CODSOR": Currently IN,
"Original_company": EUR,
"Leave_Category":Unspecified,
"Leave_Reason":Currently IN,
"Transfer_Reason":Not Transferred,
"Transfer_company":"UNSPECFIED,
"Explanation":"Contract Entry Date is 2017, he's been in the company for 7 years (2024-2017), yet, he's a classified a Manager, he should be higher in the ranks,"
 
   '''
   ,
  '''
Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID": BaseGen21,
"Seniority_previous_company(N-2) (in days)":0,
"Contract_in_group_duration":13900 days,
"Seniority_previous_company(N-1) (in days)":1481,
"Seniority_in_Group (in days)":12400,
"Contract_Date":06/08/2024,
"Leave_Date":31/12/2050,
"Entry_Date_Company_1":17/07/2020,
"Sex":Mrs,
"Contract Type":DD,
"CODENT":Mutation,
"Classification":AMA,
"Job Title":Supervisor,
"Specific_title":Supervisor,
"Work_form":Full Time,
"Entry_Category":ACTIVE,
"CODSOR": Currently IN,
"Original_company":AFA,
"Leave_Category":Transfer,
"Leave_Reason":End of Contract,
"Transfer_Reason":Organisation,
"Transfer_company":"ATX,
"Explanation":"Employee transferred after end of contract with the first company, after 4 years of experience (12400/365= 4years), and upon entry to the new company she is classified as a supervisor, she is eligible for a Manager role or an Executive,"

  ''']

In [15]:
batch_generation(L)

file 0 saved
step0 done
file 1 saved
step1 done
file 2 saved
step2 done


['1. "PersonID":"BaseGen2",\n"Seniority_previous_company(N-2) (in days)":0,\n"Contract_in_group_duration":"11000 days",\n"Seniority_previous_company(N-1) (in days)":0,\n"Seniority_in_Group (in days)":12000,\n"Contract_Date":"18/11/2020",\n"Leave_Date":"31/12/2050",\n"Entry_Date_Company_1":"01/01/1753",\n"Sex":"Mr",\n"Contract Type":"DI",\n"CODENT":"Assimilated Executive",\n"Classification":"Assimilated Executive",\n"Job Title":"Assimilated Executive",\n"Specific_title":"Assimilated Executive",\n"Work_form":"Full Time",\n"Entry_Category":"RERecruited",\n"CODSOR":"Currently IN",\n"Original_company":"TMG",\n"Leave_Category":"Unspecified",\n"Leave_Reason":"Currently IN",\n"Transfer_Reason":"Not Transferred",\n"Transfer_company":"UNSPECFIED",\n"Explanation":"Contract Entry Date is 2020, he\'s been in the company for 4 years (2024-2020),  he\'s a classified an Assimilated Executive,the rank associated to him is higher than that of his actual seniority years and experience,"\n\n2. "PersonID":

In [10]:
!rm -rf /kaggle/working/*

/usr/local/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


- 

In [ ]:
# Put your input here:
user_input = '''

Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID":"BaseGen1",
"Seniority_previous_company(N-2) (in days)":0,
"CONT_GRP_DUR_int":"12000 days",
"Seniority_previous_company(N-1) (in days)":0,
"Seniority_in_Group (in days)":11000,
"Contract_Date":"18/11/2020",
"Leave_Date":"31/12/2050",
"Entry_Date_Company_1":"01/01/1753",
"Sex":"Mr",
"Contract Type":"DI",
"CODENT":"ReRecruited",
"Classification":"CAS",
"Job Title":"Assimilated Executive",
"Specific_title":"Assimilated Executive",
"Work_form":"Part Time",
"Entry_Category":"ACTIVE",
"CODSOR":"Currently IN",
"Original_company":"TMG",
"Leave_Category":"Unspecified",
"Leave_Reason":"Currently IN",
"Transfer_Reason":"Not Transferred",
"Transfer_company":"UNSPECFIED",
"Explanation":"Contract Entry Date is 2020, he\'s been in the company for 4 years (2024-2020),  he\'s a classified an Assimilated Executive,the rank associated to him is higher than that of his actual seniority years and experience,"

Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID": BaseGen11,
"Seniority_previous_company(N-2) (in days)":0,
"CONT_GRP_DUR_int":11135 days,
"Seniority_previous_company(N-1) (in days)":0,
"Seniority_in_Group (in days)":12135,
"Contract_Date":10/10/2017,
"Leave_Date":31/12/2050,
"Entry_Date_Company_1":01/01/1753,
"Sex":Mr,
"Contract Type":DI,
"CODENT":New Recruit,
"Classification":DIR,
"Job Title":Manager/Director,
"Specific_title":Executive C3,
"Work_form":Part Time,
"Entry_Category":ACTIVE,
"CODSOR": Currently IN,
"Original_company": EUR,
"Leave_Category":Unspecified,
"Leave_Reason":Currently IN,
"Transfer_Reason":Not Transferred,
"Transfer_company":"UNSPECFIED,
"Explanation":"Contract Entry Date is 2017, he's been in the company for 7 years (2024-2017), yet, he's a classified a Manager, he should be higher in the ranks,"


Instruction : Generate  Misclassification Fraud scenarios :
Completion :
'''

#our_system_prompt="\nWe are trying to generate accounting Fraud data, in different fraud types : Employee Misclassification Fraud, Ghost Fraud, and Timesheet Fraud data, based on the examples above, try to generate one example of Misclassification fraud.\n"
initial_prompt = '''A Misclassification fraud is an accounting fraud type where the perpetrator misclassifies an employee of an enterprise in order for the employee to get higher or lower pay than what his diploma or seniority should allow him.
Generate data for a Misclassification fraud scenario, based on corresponding labels such as 'seniority in group' 'entry Date' 'Leave reason' etc. Nota that the actual year we are currently at is : 2024. '''

prompt = f"<s>[INST] <<SYS>>{initial_prompt}<</SYS>>\n\n{user_input} [/INST]"


inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids.to(model_new.device)
outputs =model_new.generate(input_ids=inputs, 
                             max_length=32768, #32768  #16384  #8192  #4096
                             do_sample=True,
                             temperature=1.4
                            )[0]

answer_start = int(inputs.shape[-1])
pred2 = tokenizer.decode(outputs[answer_start:], skip_special_tokens=True)

print(f'### User Input:\n{user_input}\n\n### Assistant Output:\n{pred2}')


In [ ]:
# save str in file format
with open( 'Assisant_output2.txt', 'w') as f:
    f.write(pred2)
print("file saved")

# save to csv


In [ ]:
Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID": BaseGen11,
"Seniority_previous_company(N-2) (in days)":0,
"CONT_GRP_DUR_int":11135 days,
"Seniority_previous_company(N-1) (in days)":0,
"Seniority_in_Group (in days)":12135,
"Contract_Date":10/10/2017,
"Leave_Date":31/12/2050,
"Entry_Date_Company_1":01/01/1753,
"Sex":Mr,
"Contract Type":DI,
"CODENT":New Recruit,
"Classification":DIR,
"Job Title":Manager/Director,
"Specific_title":Executive C3,
"Work_form":Part Time,
"Entry_Category":ACTIVE,
"CODSOR": Currently IN,
"Original_company": EUR,
"Leave_Category":Unspecified,
"Leave_Reason":Currently IN,
"Transfer_Reason":Not Transferred,
"Transfer_company":"UNSPECFIED,
"Explanation":"Contract Entry Date is 2017, he's been in the company for 7 years (2024-2017), yet, he's a classified a Manager, he should be higher in the ranks,"


In [ ]:
Instruction : Generate one Misclassification Fraud scenario :
Completion : 
"PersonID": BaseGen21,
"Seniority_previous_company(N-2) (in days)":0,
"CONT_GRP_DUR_int":13900 days,
"Seniority_previous_company(N-1) (in days)":1481,
"Seniority_in_Group (in days)":12400,
"Contract_Date":06/08/2024,
"Leave_Date":31/12/2050,
"Entry_Date_Company_1":17/07/2020,
"Sex":Mrs,
"Contract Type":DD,
"CODENT":Mutation,
"Classification":AMA,
"Job Title":Supervisor,
"Specific_title":Supervisor,
"Work_form":Full Time,
"Entry_Category":ACTIVE,
"CODSOR": Currently IN,
"Original_company":AFA,
"Leave_Category":Transfer,
"Leave_Reason":End of Contract,
"Transfer_Reason":Organisation,
"Transfer_company":"ATX,
"Explanation":"Employee transferred after end of contract with the first company, after 4 years of experience (12400/365= 4years), and upon entry to the new company she is classified as a supervisor, she is eligible for a Manager role or an Executive,"
